### The Normal Equation

In [ ]:
import numpy as np

X = 2 * np.random.rand(100,1)
y = 4 + 3 * X + np.random.randn(100,1)

In [ ]:
import matplotlib.pyplot as plt

#plt.scatter(X,y)
plt.plot(X,y,"b.")
plt.xlabel('X1')
plt.ylabel('y', rotation=0)
plt.axis([0,2,0,15])

plt.show()


In [ ]:
#concatenate array of ones to X
X_b = np.c_[np.ones((100,1)),X]

#Normal Equation (XT*X)^-1*(XT)*(y)
theta_best = np.linalg.inv(X_b.T.dot(X_b)).dot(X_b.T).dot(y)

print("Theta Best:", theta_best)
#our function for y was 4 + 3X +randomness
#Theta best tells us the best theta to minimize error: (theta best0) + (theta best1)X 

In [ ]:
#make some predictions using Theta Best
X_new = np.array([[0],[2]])
print("X new",X_new)

X_new_b = np.c_[np.ones((2,1)),X_new]
print("X new b",X_new_b)

y_predict = X_new_b.dot(theta_best)
print("Predicted:", y_predict)

In [ ]:
plt.plot(X_new, y_predict, "r-", label="Prediction")
plt.plot(X,y,"b.")
plt.axis([0,2,0,15])
plt.xlabel('X1')
plt.ylabel('y', rotation=0)
plt.legend()
plt.show()

In [ ]:
#Liner regression with Scikit-Learn
from sklearn.linear_model import LinearRegression

lin_reg=LinearRegression()
lin_reg.fit(X,y)
print("Intercept:", lin_reg.intercept_, "Coefficient", lin_reg.coef_)

print("X_new:",lin_reg.predict(X_new))

In [ ]:
theta_best_svd, residuals, rank, s = np.linalg.lstsq(X_b, y, rcond=1e-6)
print("Theta Best:", theta_best_svd)

print("pseudoinverse:",np.linalg.pinv(X_b).dot(y))

### Computational Complexity